In [ ]:
from pyspark.sql import SparkSession #Importamos lo necesario
from pyspark.sql.functions import *

In [ ]:
spark = SparkSession.builder.appName('CADS').getOrCreate() #Abrimos Sesión en Spark

In [ ]:
reviews = spark.read.json("../common/data/reviews.json") #Abrimos el archivo Reviews

In [ ]:
reviews = reviews.drop('helpful','summary','unixReviewTime','reviewerName') #Quitamos las columnas que no sirven
reviews = reviews.selectExpr('asin as productId', 'overall as rating', 'reviewerID as reviewerId', 'revieweText as revieweText', 'revieweTime as revieweTime') #Renombramos las columnas
reviews = reviews.fillna(value='',subset='reviewText') #Llenamos los textos nulos
reviews = reviews.withColumn("rating",reviews["rating"].cast("Integer")) #Convertimos rating a entero
reviews = reviews.withColumn("reviewTime",translate("reviewTime",",","")) #Corregimos el formato de la fecha
reviews = reviews.withColumn("reviewText",translate("reviewText",",","")) #Quitamos comas del texto
reviews = reviews.withColumn("reviewText",translate("reviewText",".","")) #Quitamos los puntos del texto
reviews = reviews.withColumn("reviewTime",translate("reviewTime"," ","-")) #Reformateamos la fecha
reviews = reviews.withColumn("reviewTime",to_date(col("reviewTime"),"MM-d-yyyy")) #Convertimos la fecha a fecha
reviews = reviews.na.drop() #Quitamos los registros con valores nulos

In [ ]:
reviews.write.json('../common/data/reviews_etl') #Guardamos el DataFrame

In [ ]:
dict_null = {col:reviews.filter(reviews[col].isNull()).count() for col in reviews.columns} #Armamos un diccionario con las cuentas de nulos
dict_null #Vemos la cuenta de nulos
reviews = reviews.na.drop() #Quitamos los registros con valores nulos

In [ ]:
reviews.printSchema() #df.info()
reviews.show() #df.head(20)